This example simulates the build up of activate products within a material under neutron irradiation. The subsequent decay of unstable isotopes is also simulated.

This first cell imports the packages needed, note the extra import openmc.deplete import

In [1]:
# remove any old files
!rm settings.xm model.xml materials.xml geometry.xml settings.xml

import openmc
import openmc.deplete
from pathlib import Path

# users might want to change these to use specific xml files to use particular decay data or transport cross sections
# the chain file was downloaded with
# pip install openmc_data
# download_endf_chain -r b8.0
chain_file = '/nuclear_data/chain-endf-b8.0.xml'
openmc.config['chain_file'] = chain_file


rm: cannot remove 'settings.xm': No such file or directory
rm: cannot remove 'geometry.xml': No such file or directory
rm: cannot remove 'settings.xml': No such file or directory
Using default filename chain-endf-b8.0.xml
     16384  [0.06%     32768  [0.12%     49152  [0.18%     65536  [0.24%     81920  [0.30%     98304  [0.36%    114688  [0.41%    131072  [0.47%    147456  [0.53%    163840  [0.59%    180224  [0.65%    196608  [0.71%    212992  [0.77%    229376  [0.83%    245760  [0.89%    262144  [0.95%    278528  [1.01%    294912  [1.07%    311296  [1.13%    327680  [1.18%    344064  [1.24%    360448  [1.30%    376832  [1.36%    393216  [1.42%    409600  [1.48%    425984  [1.54%    442368  [1.60%    458752  [1.66%    475136  [1.72%    491520  [1.78%    507904  [1.84%    524288  [1.90%    540672  [1.95%    557056  [2.01%    573440  [2.07%    589824  [2.13%    606208  [2.19%    622592  [2.25%    638976  [2.31%    655360  [2.37%    671744  [2.43%    688128  [2.49%    704512  [2.55%    

0

This section creates the geometry and the cells.
Note that it it necessary to set the volume of the material or cell.
This is so that the depletion code can find the number of atoms within the cell given the material composition, material density and volume.

In [ ]:

import math

# MATERIALS

# makes a simple material from Silver
my_material = openmc.Material() 
my_material.add_element('Ag', 1, percent_type='ao')
my_material.set_density('g/cm3', 10.49)


sphere_radius = 100
volume_of_sphere = (4/3) * math.pi * math.pow(sphere_radius, 3)
my_material.volume = volume_of_sphere  # a volume is needed so openmc can find the number of atoms in the cell/material
my_material.depletable = True  # depletable = True is needed to tell openmc to update the material with each time step

materials = openmc.Materials([my_material])
materials.export_to_xml()


# GEOMETRY

# surfaces
sph1 = openmc.Sphere(r=sphere_radius, boundary_type='vacuum')

# cells, makes a simple sphere cell
shield_cell = openmc.Cell(region=-sph1)
shield_cell.fill = my_material

# sets the geometry to the universe that contains just the one cell
geometry = openmc.Geometry([shield_cell])



This section defines the neutron source term to use and the settings

In [ ]:
# creates a 14MeV neutron point source
source = openmc.Source()
source.space = openmc.stats.Point((0, 0, 0))
source.angle = openmc.stats.Isotropic()
source.energy = openmc.stats.Discrete([14e6], [1])
source.particles = 'neutron'

# SETTINGS

# Instantiate a Settings object
settings = openmc.Settings()
settings.batches = 2
settings.inactive = 0
settings.particles = 10000
settings.source = source
settings.run_mode = 'fixed source'

model = openmc.model.Model(geometry, materials, settings)

This is the depletion specific part of the model setup.

This section specifies the chain file, this tells openmc the decay paths between isotopes including probabilities of different routes and half lives

In [ ]:
operator = openmc.deplete.CoupledOperator(
    model=model,
    normalization_mode="source-rate",  # set for fixed source simulation, otherwise defaults to fission simulation
    reduce_chain=True,  # reduced to only the isotopes present in depletable materials and their possible progeny
    reduce_chain_level=5,
)

This next stage sets the time steps and corresponding source rates for the irradiation schedule.

An output file will be produced with showing the material composition at every time step.

We are irradiating the Silver for multiple half lives to show build up and saturation

Saturation happens when decay is = to creation of the particular isotope

Ag110 half life is 24 seconds so it will start to become saturated after 120 seconds

Ag108 half life is 145 seconds so it will not be saturated

In [ ]:
# We define timesteps together with the source rate to make it clearer
timesteps_and_source_rates = [
    (24, 1e20),
    (24, 1e20),
    (24, 1e20),
    (24, 1e20),
    (24, 1e20),  # should saturate Ag110 here as it has been irradiated for over 5 halflives
    (24, 1e20),
    (24, 1e20),
    (24, 1e20),
    (24, 1e20),
    (24, 0),
    (24, 0),
    (24, 0),
    (24, 0),
    (24, 0),
    (24, 0),
    (24, 0),
    (24, 0),
    (24, 0),
    (24, 0),
    (24, 0),
]

# Uses list Python comprehension to get the timesteps and source_rates separately
timesteps = [item[0] for item in timesteps_and_source_rates]
source_rates = [item[1] for item in timesteps_and_source_rates]


# PredictorIntegrator has been selected as the depletion operator for this example as it is a fast first order Integrator
# OpenMC offers several time-integration algorithms https://docs.openmc.org/en/stable/pythonapi/deplete.html#primary-api\n",
# CF4Integrator should normally be selected as it appears to be the most accurate https://dspace.mit.edu/handle/1721.1/113721\n",
integrator = openmc.deplete.PredictorIntegrator(
    operator=operator,
    timesteps=timesteps,
    source_rates=source_rates
)


This next section starts the depletion simulation and produces the output files

In [ ]:
integrator.integrate()

# bash command to show the output files produce
!ls -lh *.h5

This section extracts the results of the depletion simulation from the h5 file and gets the amount of Ag110 in the material at each of the time steps

In [ ]:
results = openmc.deplete.ResultsList.from_hdf5("depletion_results.h5")

times, number_of_Ag110_atoms = results.get_atoms(my_material, 'Ag110')

for time, num in zip(times, number_of_Ag110_atoms):
    print(f" Time {time}s. Number of Ag110 atoms {num}")

In addition to Ag110 other atoms get created. This section plots the number of nuclides in the material excluding the original nuclides in the unirradiated material

In [ ]:
import openmc_depletion_plotter
# this package provides convenient plotting methods for depletion simulations like this one
# more details here https://github.com/fusion-energy/openmc_depletion_plotter

results.plot_atoms_vs_time(excluded_material=my_material)

Not all nuclide are unstable and the unstable ones have a different half life. This next plot shows the specific activity (activity per unit mass) as a function of time.

This is useful for identifying a suitable waste repository for activated waste.

In [ ]:
results.plot_activity_vs_time()